In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold
from xgboost.sklearn import XGBClassifier
import xgboost as xgb
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix

In [4]:
diabetic7 = pd.read_csv('data/final_data.csv')

In [5]:
diabetic7.shape

In [6]:
diabetic7.columns

In [7]:
diabetic7 = diabetic7.drop(['admission_type_id'], axis=1)

In [8]:
diabetic7.dtypes

In [9]:
le = preprocessing.LabelEncoder()

In [10]:
categorical = diabetic7.select_dtypes(include = ["object"]).columns
numerical = diabetic7.select_dtypes(exclude = ["object"]).columns
print("Number of Categorical features: " + str(len(categorical)))
print("Number of Numerical features: " + str(len(numerical)))
diabetic7_categorical = diabetic7[categorical]
diabetic7_numerical = diabetic7[numerical]

In [11]:
categorical= list(categorical)
numerical = list(numerical)

In [12]:
for i in categorical:
    diabetic7[i] = le.fit_transform(diabetic7[i])

In [13]:
diabetic7.dtypes

In [14]:
diabetic7.columns

In [15]:
answer = diabetic7['target']
diabetic7 = diabetic7.drop(['target'], axis=1)

In [16]:
#divide the dataset into train/test 7:3?
X_train, X_test, y_train, y_test = train_test_split(diabetic7, answer, test_size = 0.3, random_state = 0, stratify=answer, shuffle=True)
print("X_train : " + str(X_train.shape))
print("X_test : " + str(X_test.shape))
print("y_train : " + str(y_train.shape))
print("y_test : " + str(y_test.shape))

In [17]:
model=XGBClassifier(random_state=0, class_weight = {0:1, 1:8})

In [18]:
n_trees_range = range(10,20, 4)  # OOB score will warning if too few trees
train_error2 = []
test_error2 = []
oob_error = []

for n_trees in n_trees_range:
    model.set_params(n_estimators=n_trees, random_state=42, scale_pos_weight = 8, max_depth=6, n_jobs = -1)
    model.fit(X_train, y_train)
    train_error2.append(1 - model.score(X_train, y_train))
    test_error2.append(1 - model.score(X_test, y_test))

In [19]:
model.get_params

In [20]:
import matplotlib.pyplot as plt
plt.plot(n_trees_range, train_error2, c='red', label='training error')
plt.plot(n_trees_range, test_error2, c='blue', label='test error')
plt.ylabel('Error')
plt.xlabel('Number of trees')
plt.legend()
plt.show()

In [21]:
confusion_matrix(y_train, model.predict(X_train))

In [22]:
{'reg_alpha': 0.2,
 'min_samples_split': 10,
 'min_samples_leaf': 4,
 'max_depth': 110,
 'learning_rate': 0.1,

 'subsample': 0.5,
 'n_estimators': 400,
 'min_child_weight': 6,
 'gamma': 0.2,
 'colsample_bytree': 0.5}

In [259]:
#randomized searchcv to find the range of the hyperparameters of the XGboost


cv = StratifiedKFold(shuffle  = True, n_splits = 4)
n_estimators = [12, 14, 16]
min_samples_split = [2, 5, 10]
min_samples_leaf = min_samples_leaf = [1, 2, 4]
min_child_weight = [1,6,12]
colsample_bytree = [0.1, 0.5, 0.9]
gamma = [0, 0.2, 0.5, 0.8]
learning_rate =  [0.02, 0.06, 0.1] # default 0.1 
max_depth =  [int(x) for x in np.linspace(10, 110, num = 11)]# default 3
subsample =  [0.2, 0.5, 0.9]
reg_alpha=[0.2, 0.4, 0.7, 0.9] 

random_para = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'gamma':gamma,
               'min_child_weight': min_child_weight,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'learning_rate': learning_rate,
               'subsample': subsample,
               'colsample_bytree': colsample_bytree,
               'reg_alpha': reg_alpha,
               'njobs': [-1]}

rand_search_rf = RandomizedSearchCV(model, random_para, cv=cv)
rand_search_rf.fit(X_train, y_train)


RandomizedSearchCV(cv=StratifiedKFold(n_splits=4, random_state=None, shuffle=True),
          error_score='raise-deprecating',
          estimator=XGBClassifier(base_score=0.5, booster='gbtree', class_weight={0: 1, 1: 8},
       colsample_bylevel=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1),
          fit_params=None, iid='warn', n_iter=10, n_jobs=None,
          param_distributions={'n_estimators': [12, 14, 16], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110], 'gamma': [0, 0.2, 0.5, 0.8], 'min_child_weight': [1, 6, 12], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'learning_rate': [0.02, 0.06, 0.1], 'subsample': [0.2, 0.5, 0.9], 'colsample_bytree': [0.1, 0.5, 0.9], 'reg_alpha': [0.2

In [ ]:
#taking risks scores for variables/groups
#take out a hold-out sample of observations
#risk-table - for categories 
#risk features

In [ ]:
#auc - validation easily feasible
#true positive/false positive rates 
#classify a group as risk group -> average of highest and lowest risk groups

In [260]:
rand_search_rf.best_params_

{'subsample': 0.9,
 'reg_alpha': 0.4,
 'njobs': -1,
 'n_estimators': 16,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'min_child_weight': 6,
 'max_depth': 10,
 'learning_rate': 0.06,
 'gamma': 0.8,
 'colsample_bytree': 0.9}

In [8]:
#fit the model and get auc score

In [261]:
#randomized searchcv to find the range of the hyperparameters of the XGboost

cv = StratifiedKFold(shuffle  = True, n_splits = 4)
n_estimators = [14,16,18]
min_child_weight = [1,6,12]
colsample_bytree = [0.9, 0.95,0.99]
gamma = [0.8, 0.9]
learning_rate =  [0.05, 0.06, 0.07] # default 0.1 
max_depth =  range(10,20)# default 3
subsample =  [0.9, 0.99]
reg_alpha=[0.2, 0.4, 0.7, 0.9] 

random_para = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'gamma':gamma,
               'min_child_weight': min_child_weight,
               'min_samples_split': [5],
               'min_samples_leaf': [2],
               'learning_rate': learning_rate,
               'subsample': subsample,
               'colsample_bytree': colsample_bytree,
               'reg_alpha': [0.4],
               'njobs': [-1]}

rand_search_rf = RandomizedSearchCV(model, random_para, cv=cv)
rand_search_rf.fit(X_train, y_train)
rand_search_rf.best_params_

{'subsample': 0.9,
 'reg_alpha': 0.4,
 'njobs': -1,
 'n_estimators': 16,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'min_child_weight': 6,
 'max_depth': 17,
 'learning_rate': 0.06,
 'gamma': 0.8,
 'colsample_bytree': 0.9}

In [265]:
###Grid Search
#better practice would be to run randomized searchcv first to narrow down the range
#then run gridsearchcv

#import stratified K fold 
cv = StratifiedKFold(shuffle  = True, n_splits = 4)
min_samples_split = [4,5,6]
min_samples_leaf = [1,2]
min_child_weight = [5,6,7]
colsample_bytree = [0.4,0.5,0.6]
gamma = [0.8]
learning_rate =  [0.06] # default 0.1 
max_depth =  [17] # default 3
n_estimators = [16] # default 100
subsample =  [0.9]
reg_alpha=[0.4] 
grid_para = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'gamma':gamma,
               'min_child_weight': min_child_weight,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'subsample': subsample,
               'colsample_bytree': colsample_bytree,
               'reg_alpha': reg_alpha,
               'njobs': [-1]}
#import stratified K fold 
grid_search_forest = GridSearchCV(model, grid_para, cv=cv , scoring = 'roc_auc') #or roc_auc for scoring
grid_search_forest.fit(X_train, y_train)

bestparam= grid_search_forest.best_params_
bestscore= grid_search_forest.best_score_

In [266]:
bestparam

{'colsample_bytree': 0.6,
 'gamma': 0.8,
 'max_depth': 17,
 'min_child_weight': 7,
 'min_samples_leaf': 1,
 'min_samples_split': 4,
 'n_estimators': 16,
 'njobs': -1,
 'reg_alpha': 0.4,
 'subsample': 0.9}

In [41]:
# Fit the model
model=XGBClassifier(random_state = 0,colsample_bytree = 0.6,
 gamma = 0.8,
 max_depth = 17,
 min_child_weight= 7,
 min_samples_leaf= 1,
 min_samples_split= 4,
 n_estimators= 16,
 njobs= -1,
 reg_alpha= 0.4,
 subsample= 0.9)
#model.set_params(bestparam)
model.fit(X_train, y_train)
model.score(X_test, y_test)

In [40]:
bestparam

In [34]:
from sklearn.metrics import roc_auc_score
AUC = roc_auc_score(y_test, model.predict(X_test))

In [39]:
bestscore

In [36]:
AUC

In [37]:
confusion_matrix(y_test, model.predict(X_test))